In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import pandas as pd
import seaborn as sns
import pickle
import random
import os

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
with open(os.path.join("dataset", "train.p"), mode='rb') as training_data:
    train = pickle.load(training_data)
with open(os.path.join("dataset", "valid.p"), mode='rb') as validation_data:
    valid = pickle.load(validation_data)

X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']

In [ ]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train)
X_valid, y_valid = shuffle(X_valid, y_valid)
# X_train, y_train = X_train[:500], y_train[:500]

In [ ]:
# Normalize image to [0, 1]
X_train_norm = X_train / 255
X_valid_norm = X_valid / 255

In [ ]:
import optuna
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from optuna.integration import TFKerasPruningCallback

# Ensure TensorFlow uses GPU memory efficiently
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)


def objective(trial):
    num_filters = trial.suggest_categorical('num_filters', [32, 64, 128])
    epochs = trial.suggest_int('epochs', 2, 10)
    kernel_size = trial.suggest_categorical('kernel_size', [3, 5])
    pool_size = trial.suggest_int('pool_size', 2, 3)
    padding = trial.suggest_categorical('padding', ['same'])
    conv_iterations = trial.suggest_int('conv_iterations', 1, 3)
    compound_iterations = trial.suggest_int('compound_iterations', 1, 3)
    dense_layer_iterations = trial.suggest_int('dense_layer_iterations', 1, 2)
    strides = trial.suggest_int('strides', 1, 2)
    dropout = trial.suggest_float('dropout', 0.1, 0.5, step=0.1)
    last_layer_number = trial.suggest_categorical('last_layer_number', [43, 64, 128])
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])

    model = models.Sequential()
    model.add(layers.Conv2D(filters=num_filters, kernel_size=kernel_size, padding=padding,
                            activation="relu", input_shape=(32, 32, 3)))

    for _ in range(compound_iterations):
        for _ in range(conv_iterations):
            model.add(layers.Conv2D(filters=num_filters, kernel_size=kernel_size, padding=padding, activation="relu"))
        model.add(layers.MaxPool2D(pool_size=pool_size, strides=strides, padding='same'))
        model.add(layers.Dropout(dropout))

    model.add(layers.Flatten())
    for _ in range(dense_layer_iterations):
        model.add(layers.Dense(64, activation='relu'))

    model.add(layers.Dense(last_layer_number, activation='softmax'))

    # Compile the model
    model.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=3,
        verbose=1,
        restore_best_weights=True
    )

    pruning_callback = TFKerasPruningCallback(trial, 'val_accuracy')

    history = model.fit(
        X_train_norm, y_train,
        epochs=epochs,
        validation_data=(X_valid_norm, y_valid),
        batch_size=batch_size,
        verbose=0,
        callbacks=[early_stopping, pruning_callback]
    )

    val_accuracy = history.history["val_accuracy"][-1]
    return val_accuracy


# Use Optuna's RDBStorage for persistence
storage = optuna.storages.RDBStorage(url='sqlite:///optuna_study.db')
study = optuna.create_study(study_name='cnn_optimization2', storage=storage, direction='maximize', load_if_exists=True)
study.optimize(objective, n_trials=300)  # n_jobs for parallel execution

print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")
